# 新闻场景截图的分类

## 导入全局代码

In [ ]:
from IPython.display import display

import time
import os
import copy
import sys
import random
import math
from glob import glob
from datetime import timedelta

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import skimage.io
import matplotlib.pyplot as plt
import pandas as pd
import holoviews as hv
import hvplot.pandas

from tqdm import tqdm


## 常量定义

### 普通常量

In [ ]:
import sys

DATA_DIR = '../data/images/dev'
BATCH_SIZE = 4
NUM_EPOCHS = 60
LEARNING_RATE = 1e-5
INPUT_SIZE = 224
DATA_PARALLEL = False

phases = ['train', 'val']

# Flag for feature extracting.
#   - When False, we finetune the whole model,
#   - when True we only update the reshaped layer params
feature_extract = True

# 模型种类，输出路径
model_export_dir = 'out'
model_name = "squeezenet1_0"
model_export_file = os.path.join(model_export_dir, '{0}-{1}-py{2[0]}{2[1]}.pkl'.format(model_name, INPUT_SIZE, sys.version_info))

### 图形变形器

`torchvision` 预训练模型的的数据相当的“硬核”，其转换方式如下:

> **ℹ Tips:**
>
> 使用 [`torchvision.datasets.ImageFolder`](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder) 这个类

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]),
}

## 加载训练数据集

### ImageFolder 加载和分割

In [ ]:
def decompose_int_by_rates(i, percents):
    if not isinstance(i, int):
        raise TypeError(f'Expected type of argument `i` is `int`, but `{type(i)}` actually')
    if i < 0:
        raise ValueError('Argument `i` can not be smaller than zero')
    if sum(percents) >= 1.0:
        raise ValueError('Sum of percents can not be greater than or equal to `1.0`')
    result = []
    for percent in percents:
        result.append(math.trunc(i * percent))
    result.append(i - sum(result))
    return result

print("初始化：数据集 数据加载器 ...")

# Create training and validation datasets
images_dataset = datasets.ImageFolder(DATA_DIR)
lengths = decompose_int_by_rates(len(images_dataset), [.9,])
datasets_dict = {
    phase: subset
    for phase, subset in zip(phases, random_split(images_dataset, lengths))
}
for phase, sebset in datasets_dict.items():
    sebset.dataset.transform = data_transforms[phase]

display(pd.DataFrame(
    {'name': x, 'size': len(subset)}
    for x, subset in datasets_dict.items()
))

# Create training and validation dataloaders
dataloaders_dict = {
    x: DataLoader(
        datasets_dict[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4
    )
    for x in phases
}

classes = images_dataset.classes
num_classes = len(classes)

display(pd.DataFrame({
    '图片分类': classes
}))

### torchvision 张量转 matplotlib 数组

In [ ]:
def torchim_show(image_tensor, show=True, pause=0.001):
    pause = float(pause)
    image_tensor_denormalized = (image_tensor + 1) / 2
    image_array = image_tensor_denormalized.numpy().transpose(1, 2, 0)
    if show:
        plt.imshow(image_array)
        if pause > 0:
            plt.pause(pause)
    return image_array

### 显示一批训练数据

In [ ]:
for inputs, labels in dataloaders_dict['train']:
    batch_size = inputs.size(0)
    for i in range(batch_size):
        label = labels[i].item()
        img = inputs[i]
        print(classes[label])
        torchim_show(img)
    break

## GPU / CPU 设备

就像将Tensor转到GPU一样，可将神经网络转到GPU。

如果有可用的CUDA，我们首先获取第一个可见的cuda设备：

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

## 帮助函数定义

### 模型的训练/验证函数

In [ ]:
def train_model(model, device, dataloaders, criterion, optimizer, epochs, is_inception=False, predict_value_threshold=None):
    since = time.time()

    history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = -1
    
    for epoch in range(epochs):
        epoch_desc = f'Epoch {epoch+1}/{epochs}'
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0
            
            # detect batch size
            for inputs, labels in dataloaders[phase]:
                batch_size = inputs.size(0)
                break

            # Iterate over data.
            for i, (inputs, labels) in tqdm(
                enumerate(dataloaders[phase]),
                desc=f'{epoch_desc} - {phase}',
                total=math.ceil(len(dataloaders[phase].dataset) / batch_size),
                unit='batch(s)'
            ):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    pred_values, pred_indeces = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += sum(
                    pred_index == label
                    for pred_value, pred_index, label
                    in zip(*(m.tolist() for m in (pred_values, pred_indeces, labels)))
                    if (predict_value_threshold is None) or (pred_value >= predict_value_threshold)
                )
               
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            history.append({
                'epoch': epoch,
                'phase': phase,
                'acc': epoch_acc,
                'loss': epoch_loss,
            })

            # deep copy the model
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_epoch = epoch
                    best_model_wts = copy.deepcopy(model.state_dict())
                best_acc_check_symbol = f"{'✔' if best_epoch == epoch else '✖'}"
                print(
                    f'{epoch_desc} - {phase}  Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}  best({best_acc_check_symbol})'
                )
            else:
                print(
                    f'{epoch_desc} - {phase}  Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}'
                )
        print()

    print(f'Training complete in {timedelta(seconds=time.time() - since)}')
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history


### 设置模型参数的 `.requires_grad` 属性

设置为 `False` 表示正在进行 **feature extracting**。

一般来说，预训练模型的所有参数均是 `.requires_grad=True`。

但是我们在  **feature extracting** 时，只需要对新初始化的层进行梯度计算，所以要让这些参数与要计算的梯度无关。

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## 神经网络的初始化和改造

### 定义模型的初始化函数

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = INPUT_SIZE

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = INPUT_SIZE

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = INPUT_SIZE

    elif model_name == "squeezenet1_0":
        """ Squeezenet 1.0
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = INPUT_SIZE


    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

### 执行模型的初始化函数

In [ ]:
is_inception = model_name=="inception"

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

if device != 'cpu':
    # parallel model
    if DATA_PARALLEL:
        model_ft = nn.DataParallel(model_ft)
    # Send the model to GPU if have one
    model_ft = model_ft.to(device)

print('Model: ', model_ft)

## 优化器

In [ ]:
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=LEARNING_RATE, momentum=0.9)

## 损失函数

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

## 执行训练和验证步骤

### 执行训练过程

In [ ]:
# Train and evaluate
model_ft, hist = train_model(model_ft, device, dataloaders_dict, criterion, optimizer_ft, NUM_EPOCHS, is_inception, predict_value_threshold=6.0)

### 绘图

In [ ]:
df = pd.DataFrame(hist)

df_val_hist = df.loc[df['phase']=='val']

plot_acc = df.hvplot.line(x='epoch', y='acc', by='phase').opts(title='acc')
plot_best_val_acc = df_val_hist.loc[df_val_hist.index == df_val_hist['acc'].idxmax()] \
    .hvplot.scatter(x='epoch', y='acc', size=100, alpha=0.5)
plot_loss = df.hvplot.line(x='epoch', y='loss', by='phase').opts(title='loss')


hv.Layout(plot_acc * plot_best_val_acc + plot_loss).cols(1)

### 保存模型到文件

In [ ]:
os.makedirs(model_export_dir, exist_ok=True)

print('save model_ft to: ', model_export_file)

torch.save(model_ft, model_export_file)

## 从文件载入模型

> **❗ Note:**
>
> 如果不想重新训练，且之前已经保存了模型，可以从这里开始！

In [ ]:
if device.type == 'cpu':
    model_ft = torch.load(model_export_file, map_location='cpu')
    if isinstance(model_ft, nn.DataParallel):
        model_ft = model_ft.module
else:
    if torch.cuda.device_count() > 1:
        model_ft = torch.load(model_export_file)
        model_ft = model_ft.to(device)

print(model_ft)

## 预测一个图片试试看

In [ ]:
def predict_image(model, device, image_tensor):
    with torch.no_grad():
        model = model.eval()
        batched_samples = torch.tensor([np.array(image_tensor)]).to(device)    
        outputs = model(batched_samples)
    return [m.item() for m in outputs[0]]


In [ ]:
def predict_image_by_url(url):
    image = skimage.io.imread(url)
    tsfrm = data_transforms['val']
    sample = tsfrm(transforms.ToPILImage()(image))
#     skimage.io.imshow(image)
#     plt.pause(0.001)
    torchim_show(sample)
    plt.pause(0.001)
    scores = predict_image(model_ft, device, sample)
    sum_prob = sum(scores)
    display(
        pd.DataFrame(
            {'class': s, 'score': score, 'prob': score/sum_prob}
            for s, score in zip(classes, scores)
        ).sort_values(by=['score'], ascending=False)
    )


### 预测人工选择的数据

In [ ]:
dir_path = '/home/Public/jxdaily-scenes-split/data/images/湖口/20190520.eval/'

files = [
    # 无字幕
    '湖口-20190520-66_3.jpg',
    # 有字幕
    '湖口-20190520-23_3.jpg',
    # '播音室'
    '湖口-20190520-19_1.jpg',
]

for fn in files:
    file_path = os.path.join(dir_path, fn)
    predict_image_by_url(file_path)
    print('-----------------------------')

### 预测随机数据

In [ ]:
image_files = glob('/home/Public/jxdaily-scenes-split/data/images/湖口/20190520.eval/*.jpg')
files = random.choices(image_files, k=10)

ts = time.time()

for fn in files:
    predict_image_by_url(fn)
    print('-----------------------------')

print(timedelta(seconds=time.time()-ts))

### 预测域外数据

In [ ]:
# 用湖口的模型硬上，预测瑞昌的抓图:

predict_image_by_url('/home/Public/jxdaily-scenes-split/data/vlcsnap-2019-05-22-18h17m05s583.png')

In [ ]:
# 用湖口的模型硬上，预测瑞昌的抓图:

predict_image_by_url('/home/Public/jxdaily-scenes-split/data/vlcsnap-2019-05-24-16h08m29s390.png')

In [ ]:
# 用湖口的模型硬上，预测广昌的抓图:

predict_image_by_url('/home/Public/jxdaily-scenes-split/data/vlcsnap-2019-05-24-16h13m58s190.png')
predict_image_by_url('/home/Public/jxdaily-scenes-split/data/vlcsnap-2019-05-24-16h15m36s078.png')